In [1]:
from tkinter import *
from tkinter import messagebox
import joblib
from scipy.sparse import load_npz

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import gensim.utils
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv("C:/Users/Ting/Desktop/NLP/project/new_r_1.csv")
new_r = pd.DataFrame(df)

In [4]:
new_r.isnull().sum()

Unnamed: 0.4              0
Unnamed: 0.3              0
Unnamed: 0.2              0
Unnamed: 0.1              0
Unnamed: 0                0
title                     0
ingredients               0
instructions              0
unique_ingredients        0
unique_ingredients_str    0
time                      0
taste_label               0
total_minutes             0
dtype: int64

In [5]:
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import *
from tkinter import messagebox
import joblib
from scipy.sparse import load_npz

In [14]:
root =tk.Tk()

root.title("RECIPIE RECOMMENDATION SYSTEM")
# root.attributes('-fullscreen', True)
root.geometry('800x800') 
root.configure(bg='#B7A99A')
# #----------------------

label1 = tk.Label(root, text="Exploring recipes?", foreground="#504538", bg='#B7A99A',
                  font=("Papyrus", 28, "bold"), borderwidth=5)  # Adjusted the borderwidth for a more typical appearance
label1.pack(pady=60)  # Add some padding around the label for better layout

# Create and pack the entry box
entry1 = Entry(root, width=50, font=("Papyrus", 14), fg="#504538", bg='#E3DED9')
entry1.pack(pady=20)  # Add some padding around the entry for better layout

##############################################################################

title1_label= Label(root, text="",foreground="black", bg='#B7A99A',
             font=("Papyrus",18,"bold"),borderwidth=20)
ingredients1_label= Label(root,text="",foreground="black", bg='#B7A99A',
             font=("Papyrus",13,"bold"), wraplength=600)

# instructions1_label= Label(root,text="",foreground="black",
#              font=("Papyrus",12), wraplength=500)

instructions_frame = tk.Frame(root)  # 创建一个Frame来放置Text组件和Scrollbar
instructions_frame.pack(side='bottom', expand=True)

instructions_text = tk.Text(instructions_frame, wrap="word", bg='#E3DED9', font=("Papyrus", 12), height=10, state="disabled")
instructions_text.pack(side="left", expand=True)

scrollbar = tk.Scrollbar(instructions_frame, command=instructions_text.yview)
scrollbar.pack(side="right", fill="y")



# 更新指令文本的函数，适用于Text组件
def update_instructions_text(text):
    instructions_text.config(state="normal")  # 允许编辑文本
    instructions_text.delete("1.0", tk.END)  # 清空当前内容
    instructions_text.insert(tk.END, text)  # 插入新文本
    instructions_text.config(state="disabled")  # 禁止编辑

title1_label.pack()
ingredients1_label.pack()
# instructions1_label.pack()


##############################################################################

# # Correct way to load .npz files
# r_title = load_npz("r2_title.npz")
# r_ingredient = load_npz("r2_ingredient.npz")
# r_time = load_npz("r2_time.npz")

m_title = joblib.load("T_title.joblib")
m_ingredients = joblib.load("T_ingredients.joblib")
# m_time = joblib.load("T_time.joblib")

labelTF = joblib.load("labelTF.joblib")
LR = joblib.load("LR_taste_claddifier.joblib")
svdTitle = joblib.load("svdTitle.joblib")
svdIngredient = joblib.load("svdIngredient.joblib")

data_title = np.load("r2_title_reduced.npz")
r2_title_reduced = data_title['r2_title_reduced']

data_ingredient = np.load("r2_ingredient_reduced.npz")
r2_ingredient_reduced = data_ingredient['r2_ingredient_reduced']
######################################################################################

def recepie_recommendation():
    
    def count_time_similarity(rtime, t):
        # 确保rtime是数字
        if pd.isnull(rtime):
            return float(0)  # 对于空值或无效值，返回None或其他标记值
        elif float(t) == float(rtime):
            return float(1)  # 如果时间完全相同，相似度为无穷大
        else:
            return 1 / abs(float(t) - float(rtime))
        
    
    def word_or_fraction_to_number(value):
        try:
            # Check for and convert fractions
            if '/' in value:
                return float(eval(value))
            # Attempt to convert words to numbers
            return w2n.word_to_num(value)
        except:
            # Handle any exceptions (invalid input, etc.)
            return None

    def convert_time_to_minutes(time_string):
        clean_string = time_string.replace(",", "")
        parts = clean_string.split(" ")
        
        total_minutes = 0
        for i in range(len(parts)):
            # Check if the previous part is a number, word, or fraction
            prev_part_as_number = None
            try:
                prev_part_as_number = float(parts[i-1])
            except ValueError:
                # Attempt to convert from word or fraction
                prev_part_as_number = word_or_fraction_to_number(parts[i-1])
            
            # Handle cases where conversion wasn't possible
            if prev_part_as_number is None:
                print(f"Warning: Could not convert '{parts[i-1]}' to a number.")
                continue  # Skip this iteration
            
            if parts[i] in ["hours", "hour", "hrs", "hr"]:
                total_minutes += prev_part_as_number * 60
            elif parts[i] in ["minutes", "minute", "mins", "min"]:
                total_minutes += prev_part_as_number
            elif parts[i] in ["seconds", "second", "secs", "sec"]:
                total_minutes += prev_part_as_number / 60

        return total_minutes

    
    Q_raw = entry1.get()
    Q = pd.Series([Q_raw.strip().lower()])
    Q_label_vec = labelTF.transform(Q)
    pred_Q = LR.predict(Q_label_vec)

    Q_title = svdTitle.transform(m_title.transform(Q))
    Q_ingredient = svdIngredient.transform(m_ingredients.transform(Q))
    Qtime = Q.apply(convert_time_to_minutes)
    
    
    
    # Step 1: Filter new_r DataFrame
    filtered_indices = new_r[new_r['taste_label'].isin(pred_Q)].index

    # Assuming r2_title, r2_ingredient, r2_instruction are arrays or matrices with rows corresponding to new_r indices
    # Step 2: Index the necessary rows
    filtered_r2_title = r2_title_reduced[filtered_indices, :]
    filtered_r2_ingredient = r2_ingredient_reduced[filtered_indices, :]
    time_score = {}
    time_score['time_similarity'] =new_r.loc[filtered_indices, 'total_minutes'].apply(lambda x: count_time_similarity(x, Qtime))

    # Step 3: Calculate Cosine Similarity
    # Note: Q2_title, Q2_ingredient, Q2_time should be 2D arrays (e.g., shape (1, n_features)) to match sklearn's expectations
    cos_score = {}  # Initialize dictionary to store cosine similarity scores
    cos_score['title'] = cosine_similarity(Q_title, filtered_r2_title).flatten()
    cos_score['ingredient'] = cosine_similarity(Q_ingredient, filtered_r2_ingredient).flatten()
    
    
    # Example weights (you can adjust these based on your preference)
    weights = {
        'title': 0.4, # 20% weight for title
        'ingredient': 0.5, # 50% weight for ingredient
        'time': 0.1, # 30% weight for instruction
    }

    # Calculate weighted average of cosine similarity scores
    def normalize(array):
        min_val = np.min(array)
        max_val = np.max(array)
        return (array - min_val) / (max_val - min_val)

    # 对分数进行标准化
    normalized_title = normalize(cos_score['title'])
    normalized_ingredient = normalize(cos_score['ingredient'])
    normalized_time_similarity = normalize(time_score['time_similarity'].values)

    # 计算加权分数
    weighted_scores = (weights['title'] * normalized_title +
                    weights['ingredient'] * normalized_ingredient +
                    weights['time'] * normalized_time_similarity)
    # Get the indices of the recipes sorted by weighted average scores (top 10)
    top_1_indices_weighted = np.argsort(weighted_scores)[-1:][::-1]
    
    recommended_title = new_r['title'].iloc[filtered_indices[top_1_indices_weighted]].item()
    # recommended_ingredients = new_r['unique_ingredients'].iloc[filtered_indices[top_1_indices_weighted]].replace("[", "").replace("]", "")
    # Assuming `recommended_ingredients` contains the string including brackets
    ingredients_str = new_r['unique_ingredients'].iloc[filtered_indices[top_1_indices_weighted]].values[0]
    ingredients_str = ingredients_str.replace("[", "").replace("]", "").replace("'", "")

 

    recommended_instructions = new_r['instructions'].iloc[filtered_indices[top_1_indices_weighted]].item()
    # recommended_instructions = new_r['total_minutes'].iloc[filtered_indices[top_1_indices_weighted]].item()
    title1_label.config(text=f"{recommended_title}")
    ingredients1_label.config(text=f"Ingredients: {ingredients_str}")
    # instructions1_label.config(text=f"{recommended_instructions}")
    instructions_text.config(yscrollcommand=scrollbar.set)
    update_instructions_text(f"{recommended_instructions}")
    # instructions1_label.config(text=f"{recommended_instructions}")
    ##########################################################################################################################################################

my_button=Button(text="Enter",activebackground="#C9AD8D", foreground="#504538", bg='#B7A99A',
                 borderwidth=3,font=("Didot",11,"bold"), command=recepie_recommendation)
my_button.pack(side='bottom', pady=30, before=instructions_frame)

root.mainloop()

C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif float(t) == float(rtime):
C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return 1 / abs(float(t) - float(rtime))


C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif float(t) == float(rtime):
C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return 1 / abs(float(t) - float(rtime))


C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif float(t) == float(rtime):
C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return 1 / abs(float(t) - float(rtime))


C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif float(t) == float(rtime):
C:\Users\Ting\AppData\Local\Temp\ipykernel_17552\2985098100.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return 1 / abs(float(t) - float(rtime))
